In [1]:
import rioxarray
import os
import json
from tqdm import tqdm

In [2]:
CHELSA_DIR = "/shares/wegner.ics.uzh/CHELSA"
monthly_raster_names = os.listdir(CHELSA_DIR + "/climatologies/1981-2010/")
monthly_raster_names.remove("bio")
bio_files = []
for file in os.listdir(CHELSA_DIR + "/climatologies/1981-2010/bio"):
    if "bio" in file:
        bio_files.append(file)
print(monthly_raster_names)
print(len(bio_files))

['sfcWind', 'normalization_values.json', 'rsds', 'hurs', 'tas', 'cmi', 'tasmin', 'clt', 'vpd', 'pr', 'tasmax', 'pet']
19


In [5]:
import numpy as np
from sklearn.decomposition import PCA

all_encs = np.load("/home/jdolli/chelsaCLIP/src/utils/test_cases/data/satCLIP_intermediate_" + "zur" + ".npy")
pca = PCA(n_components=3).fit(all_encs)
red = pca.transform(all_encs)
print(red.shape)

(92400, 3)


In [ ]:
 ras = rioxarray.open_rasterio('/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/CHELSA_bio5_1981-2010_V.2.1.tif', cache=False)

In [ ]:
# Calculating the normalization values for all files
normalization_dict = {}
for bio in tqdm(bio_files):
    bio_file_name = CHELSA_DIR + "/climatologies/1981-2010/bio/" + bio
    bio_file = rioxarray.open_rasterio(bio_file_name)
    normalization_dict[bio_file_name] = (bio_file.mean(), bio_file.std())
    del bio_file
print(len(normalization_dict), bio_file_name, normalization_dict[bio_file_name])

 79%|███████▉  | 15/19 [02:19<00:37,  9.33s/it]


In [4]:
import torch
torch.deg2rad(torch.tensor([89,-179]))

tensor([ 1.5533, -3.1241])

In [4]:
import numpy as np
z = np.zeros((3,4))
mask = np.array([[True, True, False, False],[True, False,True, True],[False, False, True, True]])
vals = np.array([1,2,3,4,5,6,7])
z[mask] = vals
z

NameError: name 'rioxarray' is not defined

In [5]:
import numpy as np
for monthly in tqdm(monthly_raster_names[:2]):
    for mfn in os.listdir(CHELSA_DIR + "/climatologies/1981-2010/" + monthly):
        monthly_file_name = CHELSA_DIR + "/climatologies/1981-2010/" + monthly + "/" + mfn
        monthly_file = rioxarray.open_rasterio(monthly_file_name)
        print(monthly_file_name, np.array(monthly_file).max())
        #normalization_dict[monthly_file_name] = (monthly_file.mean(), monthly_file.std())
        #del monthly_file
#print(len(normalization_dict), monthly_file_name, normalization_dict[monthly_file_name])

  0%|          | 0/2 [00:00<?, ?it/s]

/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/sfcWind/CHELSA_sfcWind_03_1981-2010_V.2.1.tif 65535
/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/sfcWind/CHELSA_sfcWind_01_1981-2010_V.2.1.tif 65535
/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/sfcWind/CHELSA_sfcWind_04_1981-2010_V.2.1.tif 65535


  0%|          | 0/2 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
for key in normalization_dict.keys():
    mean, std = normalization_dict[key]
    normalization_dict[key] = (float(mean), float(std))
print(normalization_dict[key])

In [ ]:
# !! IMPORTANT NOTE: I also sued sea tiles for normalization, not only land. This might cause issues !!

with open(CHELSA_DIR + "/climatologies/1981-2010/" + "normalization_values.json", "w") as f:
    json.dump(normalization_dict, f)

In [8]:
with open(CHELSA_DIR + "/climatologies/1981-2010/" + "normalization_values.json", "r") as f:
    norm = json.load(f)

In [26]:
monthly_file = rioxarray.open_rasterio('/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010_normalized/bio/CHELSA_bio4_1981-2010_V.2.1.tif')

In [54]:
monthly_file

<xarray.DataArray (band: 1, y: 20880, x: 43200)>
array([[[1.35627 , 1.35627 , ..., 1.35627 , 1.35627 ],
        [1.35627 , 1.35627 , ..., 1.35627 , 1.35627 ],
        ...,
        [1.149944, 1.151117, ..., 1.151117, 1.151117],
        [1.151117, 1.151117, ..., 1.151117, 1.151117]]])
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 84.0 83.99 83.98 83.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [11]:
mean, std = norm['/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/bio/CHELSA_bio4_1981-2010_V.2.1.tif']

In [24]:
'/'.join('/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/bio/CHELSA_bio4_1981-2010_V.2.1.tif'.split("/")[:-1])

'/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010/bio'

In [15]:
monthly_file = (monthly_file - mean) / std

In [17]:
monthly_file.mean()

<xarray.DataArray ()>
array(6.14495447e-16)
Coordinates:
    spatial_ref  int64 0

In [25]:
monthly_file.rio.to_raster('/shares/wegner.ics.uzh/CHELSA/climatologies/1981-2010_normalized/bio/CHELSA_bio4_1981-2010_V.2.1.tif')

KeyboardInterrupt: 

In [21]:
rioxarray.open_rasterio('./test.tif').mean()

<xarray.DataArray ()>
array(6.14495447e-16)
Coordinates:
    spatial_ref  int64 0

In [28]:
# Create the train/test/val location arrays from the land-sea-mask
import math
from netCDF4 import Dataset
import numpy as np
from tqdm import tqdm

In [29]:
with rioxarray.open_rasterio(climatology_dir + "cmi" + "/CHELSA_" + "cmi" + "_" + "01" + "_1981-2010_V.2.1.tif", cache=False) as f:
    lsm = np.array(f)[0]>30000

In [31]:
land = np.where(lsm == np.max(lsm))
land = np.transpose(np.stack(land).astype("int16"))

In [36]:
land

array([[    0,     0],
       [    0,     1],
       [    0,     2],
       ...,
       [20838, 17854],
       [20838, 17855],
       [20838, 17856]], dtype=int16)

In [96]:
np.save(CHELSA_DIR + '/input/land_coordinates.npy', land)

In [95]:
land_train = land
# Take 10% of train as val
val_index = np.random.choice(len(land_train), math.floor(len(land_train)/10), replace=False)
land_val = land_train[val_index]
land_train = np.delete(land_train, val_index, 0)
# Take 50% of val (thus 5% of train) as test
test_index = np.random.choice(len(land_val), math.floor(len(land_val)/2), replace=False)
land_test = land_val[test_index]
land_val = np.delete(land_val, test_index, 0)
print(land_train.shape)
print(land_val.shape)
print(land_test.shape)

(278350327, 2)
(15463907, 2)
(15463907, 2)


In [97]:
np.save(CHELSA_DIR + '/input/land_coordinates_train.npy', land_train)
np.save(CHELSA_DIR + '/input/land_coordinates_val.npy', land_val)
np.save(CHELSA_DIR + '/input/land_coordinates_test.npy', land_test)

In [105]:
lsm = Dataset(CHELSA_DIR + '/input/landseamask.nc')

In [127]:
lsm["crs"]

<class 'netCDF4._netCDF4.Variable'>
|S1 crs()
    grid_mapping_name: latitude_longitude
    long_name: CRS definition
    longitude_of_prime_meridian: 0.0
    semi_major_axis: 6378137.0
    inverse_flattening: 298.257223563
    spatial_ref: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
    GeoTransform: -180.00013888885 0.008333333300000001 0 83.99986041515001 0 -0.008333333300000001 
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of   used

In [135]:
for bio in tqdm(bio_files):
    sample_file = CHELSA_DIR + "/climatologies/1981-2010/bio/" + bio

100%|██████████| 19/19 [00:00<00:00, 330671.27it/s]


In [136]:
sample = rioxarray.open_rasterio(sample_file)

In [157]:
sample.coords

Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 84.0 83.99 83.98 83.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0

In [159]:
sample[0, 200,200]["y"]

<xarray.DataArray 'y' ()>
array(82.32902709)
Coordinates:
    band         int64 1
    x            float64 -178.3
    y            float64 82.33
    spatial_ref  int64 0